In [1]:
import numpy as np
from scipy import linalg

# Matrix Algebra

The main package is `numpy` of course, but there are some more specialized routines in `scipy.linalg` that interest us as well.

In [2]:
A = np.array([[0.5, 0.0, 0.0], 
              [0.1, 0.1, 0.3],
              [0.0, 0.2, 0.3]])

Σ_u = np.array([[2.25, 0.00, 0.00],
                [0.00, 1.00, 0.50],
                [0.00, 0.50, 0.74]])

## Question 1

Compute eigenvalues. Easy to do with `np.linalg.eigvals`.

We see that all the eigenvalues are smaller than 1 in modulus (absolute value, since they are all real. This means the dynamics of a VAR(1) system  with transition matrix A are stable, and will smoothly reconverage to 0 following a shock (because they are real).

In [3]:
np.linalg.eigvals(A)

array([ 0.46457513, -0.06457513,  0.5       ])

## Question 2

Given matrices $D$, $E$, $F$ with shapes `(m, n)`, `(n, p)`, `(p, k)`, show that:

$$\text{vec}(DEF) = (F^T \otimes D) \text{vec}(E)$$

Where $\otimes$ is the Kronecker product

### Approach 1: Do it and verify

The only thing to be aware of is that the vec operator is `.T.ravel()`, since it decomposes matrices column-wise, while `.ravel()` unstacks row-wise

In [4]:
m, n, p, k = 10, 8, 6, 3
D = np.random.normal(size=(m, n))
E = np.random.normal(size=(n, p))
F = np.random.normal(size=(p, k))

lhs = (D @ E @ F).T.ravel()
rhs = np.kron(F.T, D) @ E.T.ravel()

np.allclose(lhs, rhs)

True

### Approach 2: Do some linear algebra

I'll admit that for things like this I just go to the Matrix Cookbook. For the equation in question, it cites Minka (2000), who in turn presents it without derivation. Like many things in linear algebra, it is probably something that falls out of the definitions without being super clear why. 

Perhaps doing a small example symbolically would be helpful

In [5]:
import sympy as sp
m, n, p, k = 4, 3, 2, 2

d_elements = sp.symbols([f'd_{i}{j}' for i in range(m) for j in range(n)])
e_elements = sp.symbols([f'e_{i}{j}' for i in range(n) for j in range(p)])
f_elements = sp.symbols([f'f_{i}{j}' for i in range(p) for j in range(k)])

D = sp.Matrix(d_elements).reshape(m, n)
E = sp.Matrix(e_elements).reshape(n, p)
F = sp.Matrix(f_elements).reshape(p, k)

In [6]:
(D @ E @ F).vec()

Matrix([
[f_00*(d_00*e_00 + d_01*e_10 + d_02*e_20) + f_10*(d_00*e_01 + d_01*e_11 + d_02*e_21)],
[f_00*(d_10*e_00 + d_11*e_10 + d_12*e_20) + f_10*(d_10*e_01 + d_11*e_11 + d_12*e_21)],
[f_00*(d_20*e_00 + d_21*e_10 + d_22*e_20) + f_10*(d_20*e_01 + d_21*e_11 + d_22*e_21)],
[f_00*(d_30*e_00 + d_31*e_10 + d_32*e_20) + f_10*(d_30*e_01 + d_31*e_11 + d_32*e_21)],
[f_01*(d_00*e_00 + d_01*e_10 + d_02*e_20) + f_11*(d_00*e_01 + d_01*e_11 + d_02*e_21)],
[f_01*(d_10*e_00 + d_11*e_10 + d_12*e_20) + f_11*(d_10*e_01 + d_11*e_11 + d_12*e_21)],
[f_01*(d_20*e_00 + d_21*e_10 + d_22*e_20) + f_11*(d_20*e_01 + d_21*e_11 + d_22*e_21)],
[f_01*(d_30*e_00 + d_31*e_10 + d_32*e_20) + f_11*(d_30*e_01 + d_31*e_11 + d_32*e_21)]])

We can see the kroneker product puts the elements of $F$ in the correct positions to be factored out after matrix multiplication with vec(E). Since we're going to multiply by the vec of E (an `(n*p, 1)` vector), we'll multiply each column by the corresponding element of vec(E) and add the rows. For the first row, this will become:

$$f_{00}(d_{00} e_{00} + d_{01}e_{10} + d_{02}e_{20})$$

and

$$f_{10}(d_{00}e_{01} + d_{01}e_{11} + d_{02}e_{21})$$

Which is exactly as above. 

I still have no idea **why** this is true (beyond the fact that it arithmetically works), but it clearly is. 

In [7]:
sp.kronecker_product(F.T, D)

Matrix([
[d_00*f_00, d_01*f_00, d_02*f_00, d_00*f_10, d_01*f_10, d_02*f_10],
[d_10*f_00, d_11*f_00, d_12*f_00, d_10*f_10, d_11*f_10, d_12*f_10],
[d_20*f_00, d_21*f_00, d_22*f_00, d_20*f_10, d_21*f_10, d_22*f_10],
[d_30*f_00, d_31*f_00, d_32*f_00, d_30*f_10, d_31*f_10, d_32*f_10],
[d_00*f_01, d_01*f_01, d_02*f_01, d_00*f_11, d_01*f_11, d_02*f_11],
[d_10*f_01, d_11*f_01, d_12*f_01, d_10*f_11, d_11*f_11, d_12*f_11],
[d_20*f_01, d_21*f_01, d_22*f_01, d_20*f_11, d_21*f_11, d_22*f_11],
[d_30*f_01, d_31*f_01, d_32*f_01, d_30*f_11, d_31*f_11, d_32*f_11]])

In [8]:
(sp.kronecker_product(F.T, D) @ E.vec()).applyfunc(lambda x: sp.collect(x, f_elements))

Matrix([
[f_00*(d_00*e_00 + d_01*e_10 + d_02*e_20) + f_10*(d_00*e_01 + d_01*e_11 + d_02*e_21)],
[f_00*(d_10*e_00 + d_11*e_10 + d_12*e_20) + f_10*(d_10*e_01 + d_11*e_11 + d_12*e_21)],
[f_00*(d_20*e_00 + d_21*e_10 + d_22*e_20) + f_10*(d_20*e_01 + d_21*e_11 + d_22*e_21)],
[f_00*(d_30*e_00 + d_31*e_10 + d_32*e_20) + f_10*(d_30*e_01 + d_31*e_11 + d_32*e_21)],
[f_01*(d_00*e_00 + d_01*e_10 + d_02*e_20) + f_11*(d_00*e_01 + d_01*e_11 + d_02*e_21)],
[f_01*(d_10*e_00 + d_11*e_10 + d_12*e_20) + f_11*(d_10*e_01 + d_11*e_11 + d_12*e_21)],
[f_01*(d_20*e_00 + d_21*e_10 + d_22*e_20) + f_11*(d_20*e_01 + d_21*e_11 + d_22*e_21)],
[f_01*(d_30*e_00 + d_31*e_10 + d_32*e_20) + f_11*(d_30*e_01 + d_31*e_11 + d_32*e_21)]])

## Question 3

Decompose $\Sigma_u$ into $W \Sigma_\epsilon W^T$, with $W^T$ lower triangular and $\Sigma_\epsilon$ diagonal.

The text says to use the Cholesky factorization, but this looks like the LDL decomposition to me? Wikipedia says they're connected!

In [9]:
# Method 1: Just do the ldl decompositon
W, Σ_e, _ = linalg.ldl(Σ_u)
np.allclose(W @ Σ_e @ W.T, Σ_u)

True

In [10]:
# Method 2: Do it by hand via cholesky decomposition
L = np.linalg.cholesky(Σ_u)

# Extract diagonal 
S = np.diag(np.diag(L))

# Compute component matrices
W = L @ np.linalg.inv(S)
Σ_e = S @ S

np.allclose(W @ Σ_e @ W.T, Σ_u)

True

## Question 4

Solve the discrete lyapunov equation:

$$ \Sigma_y = A \Sigma_y A^T + \Sigma_u $$

We note that the vec operator is linear, so we can distribute it to all terms:

$$\text{vec}(\Sigma_y) = \text{vec}( A \Sigma_y A^T) + \text{\Sigma_u}()$$

Next, use the identity above:

$$\text{vec}(\Sigma_y) = (A \otimes A) \text{vec}(\Sigma_y) + \text{vec}(\Sigma_u)$$

And we can now solve for $\vec{\Sigma_y}$ directly:

$$\text{vec}(\Sigma_y) = (I - A \otimes A)^{-1}\text{vec}(\Sigma_u)$$

### Baseline

In [11]:
Σy_baseline = linalg.solve_discrete_lyapunov(A, Σ_u)
Σy_baseline

array([[3.        , 0.16088328, 0.01892744],
       [0.16088328, 1.17231739, 0.67368324],
       [0.01892744, 0.67368324, 0.9535546 ]])

### Method 1: Direct Solution

In [12]:
vec_Σy = np.linalg.inv(np.eye(9) - np.kron(A, A)) @ Σ_u.T.ravel()
Σy_direct = vec_Σy.T.reshape((3, 3))
Σy_direct

array([[3.        , 0.16088328, 0.01892744],
       [0.16088328, 1.17231739, 0.67368324],
       [0.01892744, 0.67368324, 0.9535546 ]])

In [13]:
np.allclose(Σy_baseline, Σy_direct)

True

### Method 2: Doubling Algorithm

In [14]:
Σ_y_i = Σ_u.copy()
A_i = A.copy()

def step(Σ_y, Σ_u, A):
    Σ_y_next = Σ_y + A @ Σ_y @ A.T
    A_next = A @ A
    return Σ_y_next, A_next

maxiter = 500
for i in range(maxiter):
    Σ_y_next, A_next = step(Σ_y_i, Σ_u, A_i)
    error = np.max(np.abs(Σ_y_next - Σ_y_i))
    if error < 1e-25:
        print(f'Converged at iteration {i}')
        break
    Σ_y_i, A_i = Σ_y_next, A_next

Converged at iteration 5


In [15]:
Σ_y_i

array([[3.        , 0.16088328, 0.01892744],
       [0.16088328, 1.17231739, 0.67368324],
       [0.01892744, 0.67368324, 0.9535546 ]])

In [16]:
np.allclose(Σy_baseline, Σ_y_i)

True